# PWA for pγ → ΛK⁺π⁰ via `ComPWA`

PWA study on $p \gamma \to \Lambda K^+ \pi^0 $

In [ ]:
from __future__ import annotations
from IPython.display import Math
from qrules.particle import load_pdg
from ampform.io import aslatex, improve_latex_rendering
import matplotlib.pyplot as plt
import numpy as np

improve_latex_rendering()
particle_db = load_pdg()

## Inital and final state

### Check existing particles in `PDG`

In [ ]:
particle_db["Lambda"]

In [ ]:
particle_db["K+"]

In [ ]:
particle_db["pi0"]

In [ ]:
particle_db["gamma"]

In [ ]:
particle_db["p"]

### Add custom particle for initial state

In [ ]:
E_lab_gamma = 8.5
# E_lab_gamma = 5
m_proton = 0.938
m_0 = np.sqrt(2 * E_lab_gamma * m_proton + m_proton**2)
m_eta = 0.548
m_pi = 0.135
m_0

In [ ]:
from qrules.particle import Particle, Spin

pgamma = Particle(
    name="pgamma",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=m_0,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma

In [ ]:
pgamma2 = Particle(
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    mass=m_0,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99991,
)
pgamma2

In [ ]:
particle_db.add(pgamma)
particle_db.add(pgamma2)

### Generate Transitions

In [ ]:
import qrules
import graphviz

reaction1 = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["Lambda", "K+", "pi0"],
    # allowed_intermediate_particles=["a(2)(1320)","N","Delta"],
    allowed_interaction_types=["strong"],
    formalism="canonical-helicity",
    # formalism="helicity",
    particle_db=particle_db,
    max_angular_momentum=4,
    max_spin_magnitude=4,
    mass_conservation_factor=0,
)

In [ ]:
dot_se = qrules.io.asdot(reaction1, collapse_graphs=True)
graphviz.Source(dot_se)

## Build Helicity Amplitude Models

### Intensity

In [ ]:
import ampform

model_builder = ampform.get_builder(reaction1)
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = 0, 1, 2
model_no_dynamics = model_builder.formulate()
model_no_dynamics.intensity

### Amplitudes

The first term in the amplitude model:

In [ ]:
from ampform.io import aslatex

(symbol, expr), *_ = model.amplitudes.items()
Math(aslatex({symbol: expr}, terms_per_line=1))

In [ ]:
Math(aslatex(model_no_dynamics.amplitudes))

### Parameters

In [ ]:
Math(aslatex(model_no_dynamics.parameter_defaults))

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.dynamics.assign("pgamma", create_non_dynamic_with_ff)
for name in reaction1.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

## Reaction Info

In [ ]:
initial_state, *_ = model.reaction_info.initial_state.values()
print("Initial state:")
print("-1: ", initial_state.name)
print("Final state:")
for i, p in model.reaction_info.final_state.items():
    print(f"  {i}: {p.name}")
del initial_state

## Generate phase space sample

In [ ]:
phsp_event = 500_000
from tensorwaves.data import TFPhaseSpaceGenerator, TFUniformRealNumberGenerator

rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction1.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction1.final_state.items()},
)
phsp_momenta = phsp_generator.generate(phsp_event, rng)

In [ ]:
import numpy as np
import pandas as pd

pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in phsp_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

## Visualize kinematic variables

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="numpy",
)

In [ ]:
Math(aslatex(model.kinematic_variables))

In [ ]:
from tensorwaves.data import SympyDataTransformer

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)

In [ ]:
phsp = helicity_transformer(phsp_momenta)
list(phsp)

In [ ]:
for state_id, particle in reaction1.final_state.items():
    print(f"ID {state_id}:", particle.name)

In [ ]:
import pandas as pd

phsp_frame = pd.DataFrame(phsp)
phsp_frame

## Visualize model and intensity-based sample

In [ ]:
%config InlineBackend.figure_formats = ['svg']

resonances = sorted(
    reaction1.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(
    np.real(phsp_frame["m_01"]),
    bins=100,
    alpha=0.5,
    density=True,
    color="orange",
    # color = colors[1],
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)
ax.set_xlabel(R"$m(\Lambda K^+)\;\left[\mathrm{GeV}^2\right]$ [GeV]")
N_stars = [p for p in resonances if "N" in p.name]
for p, color in zip(N_stars, colors):
    ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(
    np.real(phsp_frame["m_12"]),
    bins=100,
    alpha=0.5,
    density=True,
    color="yellow",
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)

ax.set_xlabel(R"$m(K^+ \pi^0)$ [GeV]")

K_stars = [p for p in resonances if "K" in p.name]
for p, color in zip(K_stars, colors):
    ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(
    np.real(phsp_frame["m_02"]),
    bins=100,
    alpha=0.5,
    density=True,
    color="red",
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)
ax.set_xlabel(R"$m(\Lambda \pi^0)$ [GeV]")
Sigma_stars = [p for p in resonances if "Sigma" in p.name]
for p, color in zip(Sigma_stars, colors):
    ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['png']
fig, ax = plt.subplots()
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    # vmax=0.15,
)
ax.set_title("Phase Space Dalitz Plot")
ax.set_xlabel(R"$m^2(\Lambda K^+)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(K^+ \pi^0)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
# %config InlineBackend.figure_formats = ['png']

fig, ax = plt.subplots()
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
    weights=intensity_func(phsp),
)
ax.set_title("Model-weighted Phase space Dalitz Plot")
ax.set_xlabel(R"$m^2(\Lambda K^+)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(K^+ \pi^0)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()